In [3]:
import torch
import torch.nn as nn
import os
import stim
import numpy as np
from tqdm import tqdm
from decoders.tensor_tools import DEM_to_matrices 
from decoders import Nbp

In [4]:
circuit = stim.Circuit.generated(
                "repetition_code:memory",
                rounds=3,
                distance=3,
                after_clifford_depolarization=0.01,
                after_reset_flip_probability=0.01,
                before_measure_flip_probability=0.01,
                before_round_data_depolarization=0.01)

dem = circuit.detector_error_model(decompose_errors=True)
sampler = dem.compile_sampler()
syndromes, logical_flips, errors = sampler.sample(shots=2, return_errors=True)

syndromes = torch.from_numpy(syndromes).int()
syndromes = torch.reshape(syndromes, (len(syndromes), len(syndromes[0]), 1))
logical_flips = torch.from_numpy(logical_flips).int()
errors = torch.from_numpy(errors).int()

matrices = DEM_to_matrices(dem)

In [7]:
from ldpc.bposd_decoder import BpOsdDecoder